<font size="+2" color="blue">Additional results: experiments on cross-department association rules</font>

# Practice Session 04: Basket analysis

In [ ]:
!pip install apyori

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import gzip

from apyori import apriori

# 1. Playing with apyori

In [ ]:
# LEAVE AS-IS

def print_apyori_output (association_results, info=False, info_key=False):
    for relation_record in association_results:
        itemset = list(relation_record.items)

        # Consider only itemsets of two elements
        if len(itemset) > 1:

            print("Rules involving itemset %s" % itemset)
            support = relation_record.support

            for rules in relation_record.ordered_statistics:
                antecedent = list(rules.items_base)
                consequent = list(rules.items_add)

                if info_key:
                    antecedent = [info.loc[x][info_key] for x in antecedent]
                    consequent = [info.loc[x][info_key] for x in consequent]

                confidence = rules.confidence
                lift = rules.lift

                print("%s => %s (support=%.4f, confidence=%.2f, lift=%.2f)" %
                      (antecedent, consequent, support, confidence, lift))
            print()

<font size="+1" color="red">Replace this cell with your own example of transactions (at least 20 transactions). Execute the apriori algorithm, in which you should obtain at least <strong>two</strong> rules of the form ['A', 'B'] => ['C'], i.e., at least two rules having a 2-itemset in the antecedent and a 1-itemset in the consequent. Modify the transactions until you obtain such rules.</font>

In [ ]:
transactions = [
    ['Harry Potter', 'Hermione Granger', 'Ron Weasley'],
    ['Hermione Granger', 'Ron Weasley', 'Harry Potter', 'Neville Longbottom'],
    ['Draco Malfoy', 'Pansy Parkinson'],
    ['Harry Potter', 'Hermione Granger'],
    ['Ron Weasley', 'Harry Potter', 'Ginny Weasley'],
    ['Hermione Granger', 'Neville Longbottom'],
    ['Albus Dumbledore', 'Severus Snape'],
    ['Harry Potter', 'Albus Dumbledore'],
    ['Voldemort', 'Bellatrix Lestrange', 'Draco Malfoy'],
    ['Luna Lovegood', 'Harry Potter', 'Neville Longbottom'],
    ['Ginny Weasley', 'Ron Weasley', 'Hermione Granger'],
    ['Albus Dumbledore', 'Harry Potter'],
    ['Severus Snape', 'Lily Potter'],
    ['Hagrid', 'Harry Potter', 'Hermione Granger'],
    ['Voldemort', 'Nagini'],
    ['Neville Longbottom', 'Luna Lovegood', 'Harry Potter'],
    ['Fred Weasley', 'George Weasley', 'Ron Weasley'],
    ['Hermione Granger', 'Ginny Weasley', 'Luna Lovegood'],
    ['Harry Potter', 'Hagrid', 'Ron Weasley'],
    ['Bellatrix Lestrange', 'Draco Malfoy', 'Voldemort'],
    ['Hermione Granger', 'Harry Potter', 'Neville Longbottom'],
]


<font size="+1" color="red">Replace this cell with a markdown cell containing (1) a printout of the rules you have obtained, and (2) for each of those rules, indicate clearly how the support, confidence, and lift is calculated. Do not merely repeat the formula: indicate how each number is computed based on the transactions you provided, as if you were trying to verify that the numbers are correct.</font>

In [ ]:
results = list(apriori(transactions, min_support=0.01, min_confidence=0.4, min_lift=1.0))
print_apyori_output(results)

Rules involving itemset ['Harry Potter', 'Albus Dumbledore']
['Albus Dumbledore'] => ['Harry Potter'] (support=0.0952, confidence=0.67, lift=1.27)

Rules involving itemset ['Severus Snape', 'Albus Dumbledore']
['Severus Snape'] => ['Albus Dumbledore'] (support=0.0476, confidence=0.50, lift=3.50)

Rules involving itemset ['Bellatrix Lestrange', 'Draco Malfoy']
['Bellatrix Lestrange'] => ['Draco Malfoy'] (support=0.0952, confidence=1.00, lift=7.00)
['Draco Malfoy'] => ['Bellatrix Lestrange'] (support=0.0952, confidence=0.67, lift=7.00)

Rules involving itemset ['Voldemort', 'Bellatrix Lestrange']
['Bellatrix Lestrange'] => ['Voldemort'] (support=0.0952, confidence=1.00, lift=7.00)
['Voldemort'] => ['Bellatrix Lestrange'] (support=0.0952, confidence=0.67, lift=7.00)

Rules involving itemset ['Pansy Parkinson', 'Draco Malfoy']
['Pansy Parkinson'] => ['Draco Malfoy'] (support=0.0476, confidence=1.00, lift=7.00)

Rules involving itemset ['Voldemort', 'Draco Malfoy']
['Draco Malfoy'] => ['Vol

# 2. Load and prepare the shopping baskets

In [ ]:
# LEAVE AS-IS

# File names
INPUT_PRODUCTS = "instacart-products.csv"
INPUT_TRANSACTIONS = "instacart-transactions.csv.gz"

# Read into a dataframe
products = pd.read_csv(INPUT_PRODUCTS, delimiter=",")

# Set product_id as index, and drop column aisle_id
products = products.set_index('product_id').drop(columns=['aisle_id'])

products.head(100)

,product_name,department_id
product_id,,
1,Chocolate Sandwich Cookies,19
2,All-Seasons Salt,13
3,Robust Golden Unsweetened Oolong Tea,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,1
5,Green Chile Anytime Sauce,13
...,...,...
96,Sprinklez Confetti Fun Organic Toppings,13
97,Organic Chamomile Lemon Tea,7
98,2% Yellow American Cheese,16


## 2.1. Select by department

In [ ]:
# LEAVE AS-IS

DEPT_BAKERY = 3
DEPT_VEGGIES = 4
DEPT_ALCOHOL = 5
DEPT_WORLD = 6
DEPT_DRINKS = 7
DEPT_PETS = 8
DEPT_PHARMACY = 11
DEPT_CLEANING = 17
DEPT_BABIES = 18

<font size="+1" color="red">Replace this cell with your code for *select_from_departments*.</font>

In [ ]:
def select_from_departments(dataframe, product_ids, department_ids):
  selected_products = []

  # Loop through each product ID in the provided list
  for product_id in product_ids:
        # Check if the product's department_id matches one of the department_ids
        try:
            if dataframe.loc[product_id].department_id in department_ids:
                selected_products.append(product_id)
        except KeyError:
            # Handle case where product_id is not found in the dataframe
            continue

  return selected_products

<font size="+1" color="red">Replace this cell with code to test your function with three different test cases. Each test case is a list of items and a list of 1, 2, or 3 departments.</font>

In [ ]:
def display_products_info(products, product_ids, header):
    print(header)
    for product_id in product_ids:
        try:
            product_info = products.loc[product_id]
            print(f"{product_id} {product_info['product_name']} (dept {product_info['department_id']})")
        except KeyError:
            # Handle case where product_id is not found in the dataframe
            print(f"{product_id} Not Found")

# Example test case list of product IDs
test_product_ids = [22, 26, 45, 54, 57, 71, 111, 112]

# Departments to filter (e.g., DEPT_BAKERY and DEPT_CLEANING)
department_ids = [3, 17]

# Show input products information
display_products_info(products, test_product_ids, "Input products:")

# Run the selection function
selected_products = select_from_departments(products, test_product_ids, department_ids)

# Show selected products information
display_products_info(products, selected_products, "\nSelected products:")

Input products:
22 Fresh Breath Oral Rinse Mild Mint (dept 11)
26 Fancy Feast Trout Feast Flaked Wet Cat Food (dept 8)
45 European Cucumber (dept 4)
54 24/7 Performance Cat Litter (dept 8)
57 Flat Toothpicks (dept 17)
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17)
111 Fabric Softener, Geranium Scent (dept 17)
112 Hot Tomatillo Salsa (dept 13)

Selected products:
57 Flat Toothpicks (dept 17)
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17)
111 Fabric Softener, Geranium Scent (dept 17)


## 2.2. Read and filter transactions

<font size="+1" color="red">Replace this cell with your code to read transactions, keeping only items in DEPT_PHARMACY. Remember to stop after storing 5000 of the transactions read.</font>

In [ ]:
import csv
import gzip
#We do a general function to get
# Function to read transactions and filter by department
def read_and_filter_transactions(input_transactions, products, dept_id, max_transactions=5000):
    transactions = []
    stored_count = 0
    read_count = 0

    # Open the compressed transactions file
    with gzip.open(input_transactions, "rt") as inputfile:

        # Create a CSV reader
        reader = csv.reader(inputfile, delimiter=",")

        # Iterate through each row (transaction) in the file
        for row in reader:
            read_count += 1

            # Convert product IDs to integers
            items = [int(x) for x in row]

            # Filter the items by department
            filtered_items = select_from_departments(products, items, dept_id)

            # Store the transaction if it contains products from the selected department
            if filtered_items:
                transactions.append(filtered_items)
                stored_count += 1

            # Print progress every 1000 transactions read
            if read_count % 1000 == 0:
                print(f"Transactions read: {read_count}, Transactions stored: {stored_count}")

            # Stop after storing 5000 transactions
            if stored_count >= max_transactions:
                break

    return transactions

In [ ]:
transactions_1 = read_and_filter_transactions(INPUT_TRANSACTIONS, products, [DEPT_PHARMACY])

Transactions read: 1000, Transactions stored: 102
Transactions read: 2000, Transactions stored: 219
Transactions read: 3000, Transactions stored: 313
Transactions read: 4000, Transactions stored: 422
Transactions read: 5000, Transactions stored: 526
Transactions read: 6000, Transactions stored: 619
Transactions read: 7000, Transactions stored: 722
Transactions read: 8000, Transactions stored: 810
Transactions read: 9000, Transactions stored: 920
Transactions read: 10000, Transactions stored: 1046
Transactions read: 11000, Transactions stored: 1152
Transactions read: 12000, Transactions stored: 1245
Transactions read: 13000, Transactions stored: 1352
Transactions read: 14000, Transactions stored: 1459
Transactions read: 15000, Transactions stored: 1559
Transactions read: 16000, Transactions stored: 1663
Transactions read: 17000, Transactions stored: 1753
Transactions read: 18000, Transactions stored: 1854
Transactions read: 19000, Transactions stored: 1943
Transactions read: 20000, Tran

## 2.3. Extract association rules and comment on them

<font size="+1" color="red">Replace this cell with your code to extract association rules from the read transactions.</font>

In [ ]:
results_1 = list(apriori(transactions_1, min_support=0.001, min_confidence=0.01, min_lift=5))
print_apyori_output(results_1, products, 'product_name')

Rules involving itemset [16825, 1406]
['Vegan Nutritional Shake Sweet Vanilla Bean'] => ['Vegan Smooth Chocolate Nutritional Shake'] (support=0.0012, confidence=0.40, lift=80.00)
['Vegan Smooth Chocolate Nutritional Shake'] => ['Vegan Nutritional Shake Sweet Vanilla Bean'] (support=0.0012, confidence=0.24, lift=80.00)

Rules involving itemset [7976, 16019]
['One Plant-Based Chocolate Flavor Nutritional Shake Drink Mix'] => ['All-In-One French Vanilla Nutritional Shake Sachet'] (support=0.0010, confidence=0.31, lift=97.66)
['All-In-One French Vanilla Nutritional Shake Sachet'] => ['One Plant-Based Chocolate Flavor Nutritional Shake Drink Mix'] (support=0.0010, confidence=0.31, lift=97.66)

Rules involving itemset [27544, 39162]
['Lemon Verbena Hand Soap'] => ['Clean Day Basil Hand Soap'] (support=0.0018, confidence=0.12, lift=11.21)
['Clean Day Basil Hand Soap'] => ['Lemon Verbena Hand Soap'] (support=0.0018, confidence=0.16, lift=11.21)



<font size="+1" color="red">Replace this cell with a brief commentary on what you would recommend to the shopping app considering the extracted association rules.</font>

1. **Cross-Selling Opportunities:**

  - **Vegan Nutritional Shakes:**
  
  The high lift value (80.00) between the Vegan Nutritional Shake Sweet Vanilla Bean and the Vegan Smooth Chocolate Nutritional Shake shows a good chance for cross-selling. Even though the confidence values (0.40 and 0.24) are not very high, the lift suggests that if a customer buys one of these products, they are much more likely to buy the other compared to random chance. The app could suggest the other flavor when one is added to the cart.
  - **Plant-Based Nutritional Shakes:**
  
  For the One Plant-Based Chocolate Flavor Nutritional Shake Drink Mix and the All-In-One French Vanilla Nutritional Shake Sachet, the high lift (97.66) shows a strong connection between these products. The app could offer discounts to encourage customers to buy both products together.

2. **Bundle Offers:**

  - **Hand Soaps:**
There is a link between Lemon Verbena Hand Soap and Clean Day Basil Hand Soap. While the confidence values (0.12 and 0.16) are low, the lift (11.21) suggests these products are more likely to be bought together. The app could offer a bundle deal for both hand soaps or suggest them as related products to increase sales.

3. **Product Recommendations:**
  - Use these rules to improve product recommendations. For example, when a customer adds the Vegan Nutritional Shake Sweet Vanilla Bean to their cart, the app can suggest the Vegan Smooth Chocolate Nutritional Shake as a related product. This increases the chance that the customer will buy both.

## 2.4. Extract association rules and comment on them (other departments)

<font size="+1" color="red">Replace this cell with code to select a different set of departments (at least two, not DEPT_PHARMACY) and extract transactions again. Avoid replicating code when possible.</font>

In [ ]:
transactions_2 = read_and_filter_transactions(INPUT_TRANSACTIONS, products, [DEPT_VEGGIES,DEPT_BAKERY])

Transactions read: 1000, Transactions stored: 817
Transactions read: 2000, Transactions stored: 1600
Transactions read: 3000, Transactions stored: 2400
Transactions read: 4000, Transactions stored: 3193
Transactions read: 5000, Transactions stored: 3971
Transactions read: 6000, Transactions stored: 4749


In [ ]:
results_2 = list(apriori(transactions_2, min_support=0.001, min_confidence=0.8, min_lift=1))
print_apyori_output(results_2, products, 'product_name')

Rules involving itemset [9387, 24852, 45007]
['Granny Smith Apples', 'Organic Zucchini'] => ['Banana'] (support=0.0010, confidence=1.00, lift=5.24)

Rules involving itemset [13176, 26369, 31717]
['Organic Roma Tomato', 'Organic Cilantro'] => ['Bag of Organic Bananas'] (support=0.0010, confidence=0.83, lift=5.52)

Rules involving itemset [13176, 26940, 27966]
['Organic Large Green Asparagus', 'Organic Raspberries'] => ['Bag of Organic Bananas'] (support=0.0014, confidence=0.88, lift=5.79)

Rules involving itemset [22935, 24964, 16759]
['Organic Garlic', 'Organic Carrot Bunch'] => ['Organic Yellow Onion'] (support=0.0010, confidence=0.83, lift=18.68)

Rules involving itemset [47209, 24964, 16759]
['Organic Garlic', 'Organic Carrot Bunch'] => ['Organic Hass Avocado'] (support=0.0010, confidence=0.83, lift=9.80)

Rules involving itemset [28985, 24852, 16797]
['Michigan Organic Kale', 'Strawberries'] => ['Banana'] (support=0.0018, confidence=0.90, lift=4.71)

Rules involving itemset [24852,

<font size="+1" color="red">Replace this cell with your commentary on the obtained rules.</font>

- Bananas, especially organic ones, appear frequently as consequent items in these rules, suggesting that customers who buy certain organic fruits and vegetables are highly likely to purchase bananas as well.
- High lift values (ranging from 4.36 to 9.80) indicate strong associations and significant relationships between the itemsets, showing that these combinations are more likely to occur than random chance.
- Organic products seem to have a particularly strong association in customers' purchasing patterns, especially when combined with other organic fruits or vegetables.

In [ ]:
def print_apyori_output_diff_dept(association_results, info=False, info_key=False, dept_key='department_id'):
    for relation_record in association_results:
        itemset = list(relation_record.items)

        # Consider only itemsets of two elements
        if len(itemset) > 1:

            print("Rules involving itemset %s" % itemset)
            support = relation_record.support

            for rules in relation_record.ordered_statistics:
                antecedent = list(rules.items_base)
                consequent = list(rules.items_add)

                # Retrieve departments for antecedent and consequent products
                if info_key:
                    antecedent = [info.loc[x][info_key] for x in antecedent]
                    consequent = [info.loc[x][info_key] for x in consequent]

                antecedent_depts = [info.loc[x][dept_key] for x in rules.items_base]
                consequent_depts = [info.loc[x][dept_key] for x in rules.items_add]

                # Check if any consequent product belongs to a different department from antecedent products
                if any(c_dept not in antecedent_depts for c_dept in consequent_depts):
                    confidence = rules.confidence
                    lift = rules.lift

                    print("%s => %s (support=%.4f, confidence=%.2f, lift=%.2f)" %
                          (antecedent, consequent, support, confidence, lift))
            print()


In [ ]:
DEPT=[DEPT_BABIES, DEPT_PETS]
transactions_3 = read_and_filter_transactions(INPUT_TRANSACTIONS, products, DEPT)

Transactions read: 1000, Transactions stored: 72
Transactions read: 2000, Transactions stored: 143
Transactions read: 3000, Transactions stored: 229
Transactions read: 4000, Transactions stored: 300
Transactions read: 5000, Transactions stored: 376
Transactions read: 6000, Transactions stored: 452
Transactions read: 7000, Transactions stored: 527
Transactions read: 8000, Transactions stored: 594
Transactions read: 9000, Transactions stored: 668
Transactions read: 10000, Transactions stored: 753
Transactions read: 11000, Transactions stored: 838
Transactions read: 12000, Transactions stored: 902
Transactions read: 13000, Transactions stored: 981
Transactions read: 14000, Transactions stored: 1062
Transactions read: 15000, Transactions stored: 1146
Transactions read: 16000, Transactions stored: 1227
Transactions read: 17000, Transactions stored: 1298
Transactions read: 18000, Transactions stored: 1370
Transactions read: 19000, Transactions stored: 1428
Transactions read: 20000, Transacti

In [ ]:
results_3 = list(apriori(transactions_3, min_support=0.001, min_confidence=0.8, min_lift=1))
print_apyori_output_diff_dept(results_3, products, "product_name")

Rules involving itemset [48476, 8269]

Rules involving itemset [9593, 47605]

Rules involving itemset [15190, 12902]

Rules involving itemset [30744, 49587]

Rules involving itemset [9497, 8833, 1134]

Rules involving itemset [12020, 2923, 10036]

Rules involving itemset [5114, 14211, 29557]

Rules involving itemset [38192, 34745, 5114]

Rules involving itemset [25530, 14491, 6541]

Rules involving itemset [47719, 6891, 32743]

Rules involving itemset [7076, 17766, 10863]

Rules involving itemset [12020, 10036, 15398]

Rules involving itemset [18629, 11212, 12820]

Rules involving itemset [12020, 32989, 15398]

Rules involving itemset [22969, 46082, 12343]

Rules involving itemset [23025, 46082, 12343]

Rules involving itemset [32018, 38141, 12343]

Rules involving itemset [45495, 32018, 12343]

Rules involving itemset [32018, 46082, 12343]

Rules involving itemset [49201, 32018, 12343]

Rules involving itemset [45495, 38141, 12343]

Rules involving itemset [46082, 38141, 12343]

Rules

We don't find an itemset without departament_id in common.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>